<details><summary style="display:list-item; font-size:16px; color:blue;">Jupyter Help</summary>
    
Having trouble testing your work? Double-check that you have followed the steps below to write, run, save, and test your code!
    
[Click here for a walkthrough GIF of the steps below](https://static-assets.codecademy.com/Courses/ds-python/jupyter-help.gif)

Run all initial cells to import libraries and datasets. Then follow these steps for each question:
    
1. Add your solution to the cell with `## YOUR SOLUTION HERE ## `.
2. Run the cell by selecting the `Run` button or the `Shift`+`Enter` keys.
3. Save your work by selecting the `Save` button, the `command`+`s` keys (Mac), or `control`+`s` keys (Windows).
4. Select the `Test Work` button at the bottom left to test your work.

![Screenshot of the buttons at the top of a Jupyter Notebook. The Run and Save buttons are highlighted](https://static-assets.codecademy.com/Paths/ds-python/jupyter-buttons.png)

**Setup** Run the following cell to import libraries.


**MLP Task: Predict Hotel Cancellations Using Booking Data**

In this exercise, you will use PyTorch and neural networks to predict hotel cancellations using [real-world hotel booking data](https://www.kaggle.com/datasets/jessemostipak/hotel-booking-demand) from a resort hotel. 

The goal is to build a neural network that can predict whether a reservation will be canceled ahead of time using features like the date of the booking, the length of stay, the number of people staying, and the average daily rate.

The training dataset has already been cleaned and pre-processed. It is loaded into PyTorch dataset objects `train_dataset` and `train_loader` for batching and placed onto the GPU device.

In [5]:
import torch
import pandas as pd
import numpy as np
torch.manual_seed(42)

# Load Training + Testing data
TRAIN_CSV = "datasets/bookings_train.csv"
train_df = pd.read_csv(TRAIN_CSV)

# Training Features + Target Distribution
TARGET = "is_canceled"
train_features = [x for x in train_df.columns if x not in TARGET]
print("# of Training Features:", len(train_features))
print()

print("Training Target Distribution:")
print(train_df[TARGET].value_counts().to_frame("count").assign(pct=lambda x: (x["count"] / x["count"].sum() * 100).round(2)))
print()

# Train + Test Split (using GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

X_train = torch.from_numpy(train_df[train_features].values).float().to(device)
y_train = torch.from_numpy(train_df[TARGET].values).float().view(-1, 1).to(device)
print("Training size:", y_train.shape)

# Create TensorDataset and DataLoader
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(X_train, y_train)

batch_size = 32  
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Check device
print("Device:", device)

# of Training Features: 155

Training Target Distribution:
             count   pct
is_canceled             
0             8898  50.0
1             8898  50.0

Training size: torch.Size([17796, 1])
Device: cuda


#### Checkpoint 1/3

Construct a simple MLP with the following architecture.

**A.** Initialize the model class with parameters for the input size, hidden layer size, and output size:
- First fully connected layer `fc1`: input size → hidden size
- Second fully connected layer `fc2`: hidden size → hidden size
- Third fully connected layer `fc3`: hidden size → output output
- 
**B.** Also initialize a ReLU activation layer `relu`.

**C.** In the forward pass, apply in the following order: 
- First layer → ReLU activation → second layer → ReLU activation → third layer → output.

**D.** Instantiate the model to the variable `model` with an input feature size of `155`, a hidden layer size of `360`, and an output size of `1`.

Don't forget to run the cell and save the notebook before selecting `Test Work`! Open the `Jupyter Help` toggle at the top of the notebook for more details.

In [6]:
import torch
import torch.nn as nn
torch.manual_seed(42)

## YOUR SOLUTION HERE ##
class SimpleMLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)        
        return x
        
input_features = 155
hidden_neurons = 360
output_classes = 1

model = SimpleMLP(input_features, hidden_neurons, output_classes)

# Print model summary and number of parameters
model.to(device)
from custom_torchinfo import custom_summary
custom_summary(model, input_size=(32,155))

        Layer (type)              Output Shape         Param #
            Linear-1                 [32, 360]          56,160
              ReLU-2                 [32, 360]               0
            Linear-3                 [32, 360]         129,960
              ReLU-4                 [32, 360]               0
            Linear-5                   [32, 1]             361
         SimpleMLP-6                   [32, 1]               0
Total params: 186,481
Trainable params: 186,481
Non-trainable params: 0


#### Checkpoint 2/3

Let's initialize the loss function and optimizer for training using the `torch.optim` module.

**A.** Create an instance of the binary cross-entropy (with logits) loss function in PyTorch and save it to the variable `loss_fn`.

**B.** Create an instance of the Adam optimizer in PyTorch with a learning rate of `0.0001` and save it to the variable `optimizer`.

Don't forget to run the cell and save the notebook before selecting `Test Work`! Open the `Jupyter Help` toggle at the top of the notebook for more details.

In [7]:
import torch.optim as optim
torch.manual_seed(42)

## YOUR SOLUTION HERE ##
loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

#### Checkpoint 3/3

Now, let's create a training loop that trains the MLP on 10 epochs while keeping track of the training loss and accuracy.

**A.** Train the MLP on 10 epochs by assigning the value `10` to the variable `num_epochs`.

**B.** As you loop through each epoch, initialize the following variables to `0` to keep track of during training:
- `epoch_loss`: Training loss per epoch.
- `num_batches`: Number of batches per epoch.
- `correct`: Number of correct predictions.
- `total`: Total number of predictions.
  
**C.** Build the training section:
- Loop through the training batch inputs and labels.
- Within each training batch:
  - Obtain the logits by passing the inputs through the forward pass.
  - Compute the loss using the loss function.
  - Update the parameters through the backward pass.
  - Update the loss and number of batches per epoch.
  - Convert the logits into predicted labels

**D.** For each epoch, calculate the average loss and average accuracy. 

Print out the BCE loss and accuracy per epoch.

Don't forget to run the cell and save the notebook before selecting `Test Work`! Open the `Jupyter Help` toggle at the top of the notebook for more details.ook for more det

In [12]:
torch.manual_seed(42)
model.train()

## YOUR SOLUTION HERE ##
num_epochs = 10
for epoch in range(num_epochs):
    epoch_loss = 0
    correct = 0
    total = 0
    num_batches = 0
    
    for batch_X, batch_y in train_loader:
        # Forward pass
        logits = model(batch_X)

        # Compute loss
        loss = loss_fn(logits, batch_y)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Track loss
        epoch_loss += loss.item()
        num_batches += 1

        # Convert logits to predicted labels
        probs = torch.sigmoid(logits)
        preds = (probs >= 0.5).float()

        # Accuracy
        correct += (preds == batch_y).sum().item()
        total += batch_y.size(0)
    
    # Average metrics
    avg_loss = epoch_loss / num_batches
    accuracy = correct / total

    # Print training progress
    print(f'Epoch [{epoch+1}/{num_epochs}], BCE Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}')

Epoch [1/10], BCE Loss: 0.5398, Accuracy: 0.7315
Epoch [2/10], BCE Loss: 0.4483, Accuracy: 0.7800
Epoch [3/10], BCE Loss: 0.4227, Accuracy: 0.7953
Epoch [4/10], BCE Loss: 0.4041, Accuracy: 0.8056
Epoch [5/10], BCE Loss: 0.3856, Accuracy: 0.8175
Epoch [6/10], BCE Loss: 0.3699, Accuracy: 0.8273
Epoch [7/10], BCE Loss: 0.3550, Accuracy: 0.8358
Epoch [8/10], BCE Loss: 0.3412, Accuracy: 0.8442
Epoch [9/10], BCE Loss: 0.3288, Accuracy: 0.8520
Epoch [10/10], BCE Loss: 0.3184, Accuracy: 0.8554
